#### Install Dependencies

In [4]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 -q
!pip install openai -q
!pip install -U openai-whisper -q
!pip install git+https://github.com/openai/whisper.git -q
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git -q
!pip install cohere tiktoken -q
!pip install llmx==0.0.15a0 -q
!pip install jiwer -q
!pip install setuptools-rust -q
# !sudo apt update && sudo apt install ffmpeg    # for linux
!choco install ffmpeg                        # for windows
!pip install setuptools-rust -q
!pip install moviepy
!pip install googletrans
!pip install translate
!pip install gTTS

Chocolatey v2.2.2
Installing the following packages:
ffmpeg;#;for;windows
By installing, you accept licenses for the packages.
ffmpeg v6.0.0 already installed.
 Use --force to reinstall, specify a version to install, or try upgrade.
Unable to connect to source 'https://community.chocolatey.org/api/v2/':
 Failed to fetch results from V2 feed at 'https://community.chocolatey.org/api/v2/Packages()?$filter=(tolower(Id)%20eq%20'#')%20and%20IsLatestVersion&semVerLevel=2.0.0' with following message : Response status code does not indicate success: 400 (Bad Request).
# not installed. The package was not found with the source(s) listed.
 Source(s): 'https://community.chocolatey.org/api/v2/'
 NOTE: When you specify explicit sources, it overrides default sources.
If the package version is a prerelease and you didn't specify `--pre`,
 the package may not be found.
Please see https://docs.chocolatey.org/en-us/troubleshooting for more
 assistance.
for not installed. The package was not found with th

#### Import Dependencies

In [5]:
from translate import Translator as Dubify
from moviepy.editor import VideoFileClip, AudioFileClip, clips_array
from googletrans import Translator
from gtts import gTTS
import requests
import whisper
import os

#### Load the video

In [6]:
vid_in = "input_video.mp4"

#### Visual-to-Speech

In [7]:
vid = VideoFileClip(vid_in)
aud = vid.audio

#### Write the Audio File

In [8]:
# define the output audio path and filename
aud_out = "original_audio.mp3"

aud.write_audiofile(aud_out)

MoviePy - Writing audio in original_audio.mp3


MoviePy - Done.


#### Close the files

In [9]:
vid.close()
aud.close()

#### Load model

In [10]:
# model = whisper.load_model("base")
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [02:27<00:00, 10.3MiB/s]


#### Speech-To-Text

In [14]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("original_audio.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
# options = whisper.DecodingOptions()                       # for GPU
options = whisper.DecodingOptions(fp16 = False)            # for CPU
result = whisper.decode(model, mel, options)

# extracting only the text
speech_in  = result.text

Detected language: en


#### Exporting speech to file

In [15]:
file_path = "original_speech.txt"

# Open the file in write mode and write the text to it
with open(file_path, 'w') as file:
    file.write(speech_in)


#### Text Speech Dub

###### Eng to Hindi

In [16]:
def translate_to_hindi(text):
    translator= Dubify(to_lang="hi")
    translation = translator.translate(text)
    return translation

###### Eng to TN

In [17]:
def translate_to_tamil(text):
    translator = Dubify(to_lang="ta")
    translation = translator.translate(text)
    return translation

###### Load the text file

In [18]:
with open('original_speech.txt', 'r', encoding='utf-8') as file:
    english_content = file.read()

###### Translation method call (hindi)

In [19]:
hindi_translation = translate_to_hindi(english_content)

###### Translation mehtod call (tamil)

In [20]:
tamil_translation = translate_to_tamil(english_content)

###### Export the translated text

In [21]:
with open('speech_hindi.txt', 'w', encoding='utf-8') as hindi_file:
    hindi_file.write(hindi_translation)

with open('speech_tamil.txt', 'w', encoding='utf-8') as tamil_file:
    tamil_file.write(tamil_translation)

#### Convert speech*.txt to audio

In [22]:
def text_to_audio(text, output_file, lang):
    tts = gTTS(text, lang=lang)
    tts.save(output_file)

#### Reading speechHindi.txt

In [23]:
with open('speech_hindi.txt', 'r', encoding='utf-8') as hindi_file:
    hindi_text = hindi_file.read()

#### Reading speechTamil.txt

In [24]:
with open('speech_tamil.txt', 'r', encoding='utf-8') as tamil_file:
    tamil_text = tamil_file.read()

#### Convert speechHindi.txt to Audio

In [25]:
text_to_audio(hindi_text, 'audio_hindi.mp3', 'hi')

#### Convert speechTamil.txt to Audio

In [26]:
text_to_audio(tamil_text, 'audio_tamil.mp3', 'ta')

#### Dubbing